In [1]:
!pip install google-play-scraper
!pip install sastrawi
!pip install emoji
!pip install imbalanced-learn
!pip install nltk

In [2]:
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import nltk
import emoji
from functools import lru_cache
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
import csv
import requests
from io import StringIO

# Download resource NLTK
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to C:\Users\Rucirasatti
[nltk_data]     N\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Rucirasatti
[nltk_data]     N\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Pra-pemrosesan Data
Lakukan pra-pemrosesan data untuk membersihkan teks, menghapus duplikat, dan menerapkan langkah-langkah seperti cleaning, casefolding, slangword correction, tokenizing, stopword removal, stemming, dll.


In [3]:
# Langkah 2: Pra-pemrosesan Data
print("\nLangkah 2: Pra-pemrosesan Data")
df = pd.read_csv('ulasan_whatsapp.csv')
clean_df = df.dropna().drop_duplicates()

# Menampilkan informasi tentang DataFrame
print("Informasi DataFrame setelah pembersihan awal:")
clean_df.info()



Langkah 2: Pra-pemrosesan Data
Informasi DataFrame setelah pembersihan awal:
<class 'pandas.core.frame.DataFrame'>
Index: 62895 entries, 0 to 62999
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   content  62895 non-null  object
dtypes: object(1)
memory usage: 982.7+ KB


## Definisikan Fungsi Preprocessing
Definisikan fungsi untuk cleaning, casefolding, tokenizing, stopword removal, stemming, dan penanganan slangword.


In [4]:
# Langkah 3: Definisikan fungsi preprocessing
def convert_emojis(text):
    return emoji.demojize(text, language='en')

def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text

def normalize_repeated_chars(text):
    return re.sub(r'(.)\1+', r'\1', text)

def casefoldingText(text):
    text = text.lower()
    return text

def translate_english_words(text):
    english_to_indonesian = {
        "history": "riwayat", "video": "video", "photo": "foto", "status": "status",
        "update": "pembaruan", "bug": "kesalahan", "feature": "fitur", "chat": "obrolan",
        "story": "cerita", "blur": "buram", "hd": "kualitas tinggi"
    }
    words = text.split()
    translated_words = [english_to_indonesian.get(word.lower(), word) for word in words]
    return ' '.join(translated_words)

slangwords = {
    "@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar",
    "bgt": "banget", "maks": "maksimal", "pdhl": "padahal", "agk": "agak", "sw": "status",
    "burik": "buram", "ga": "tidak", "gak": "tidak", "gk": "tidak", "nggak": "tidak",
    "tp": "tapi", "klo": "kalau", "kl": "kalau", "udh": "sudah", "sdh": "sudah",
    "bkin": "bikin", "bkn": "bukan", "krn": "karena", "krna": "karena", "bs": "bisa",
    "blm": "belum", "sm": "sama", "sma": "sama", "gt": "gitu", "gtu": "gitu",
    "jgn": "jangan", "yg": "yang", "kpn": "kapan", "knp": "kenapa", "knpa": "kenapa",
    "org": "orang", "dr": "dari", "dri": "dari", "ttg": "tentang", "hrs": "harus",
    "lg": "lagi", "lgi": "lagi", "cm": "cuma", "cma": "cuma", "aj": "aja", "ajh": "aja",
    "doang": "saja", "bgs": "bagus", "bgus": "bagus", "skrg": "sekarang", "skg": "sekarang",
    "nnti": "nanti", "nt": "nanti", "smpe": "sampai", "smp": "sampai", "trh": "terus",
    "trs": "terus", "tpi": "tapi", "bnyk": "banyak", "bnk": "banyak", "sblm": "sebelum",
    "stlh": "setelah", "slh": "salah", "ksh": "kasih", "thx": "terima kasih", "makasih": "terima kasih",
    "vc": "video call", "vn": "voice note", "hp": "handphone", "apk": "aplikasi", "min": "admin"
}

def fix_slangwords(text):
    words = text.split()
    fixed_words = []
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
    fixed_text = ' '.join(fixed_words)
    return fixed_text

def tokenizingText(text):
    text = word_tokenize(text)
    return text

def filteringText(text):
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya', 'yaa', 'sih', 'ku', 'gaa', 'loh', 'kah', 'woi', 'woii', 'woy'])
    listStopwords.difference_update({'tidak', 'gak', 'nggak', 'bukan'})
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text

# Optimasi stemming dengan cache
factory = StemmerFactory()
stemmer = factory.create_stemmer()

@lru_cache(maxsize=10000)
def cached_stem(word):
    return stemmer.stem(word)

def stemmingText(text):
    words = text.split()
    stemmed_words = [cached_stem(word) for word in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

def toSentence(list_words):
    sentence = ' '.join(word for word in list_words)
    return sentence

def remove_repeated_words(text):
    words = text.split()
    result = []
    prev_word = None
    for word in words:
        if word != prev_word:
            result.append(word)
            prev_word = word
    return ' '.join(result)


## Terapkan Preprocessing
Terapkan langkah-langkah preprocessing pada DataFrame.


In [5]:
# Langkah 4: Terapkan preprocessing
# 1. Konversi emoticon ke teks
clean_df['text_emoticon'] = clean_df['content'].apply(convert_emojis)

# 2. Membersihkan teks
clean_df['text_clean'] = clean_df['text_emoticon'].apply(cleaningText)

# 3. Normalisasi karakter berulang
clean_df['text_normalized'] = clean_df['text_clean'].apply(normalize_repeated_chars)

# 4. Mengubah huruf menjadi huruf kecil
clean_df['text_casefoldingText'] = clean_df['text_normalized'].apply(casefoldingText)

# 5. Menerjemahkan kata-kata Inggris ke Indonesia
clean_df['text_translated'] = clean_df['text_casefoldingText'].apply(translate_english_words)

# 6. Mengganti kata-kata slang
clean_df['text_slangwords'] = clean_df['text_translated'].apply(fix_slangwords)

# 7. Memecah teks menjadi token
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)

# 8. Menghapus stopwords
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)

# 9. Menggabungkan token menjadi kalimat
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

# 10. Stemming
# clean_df['text_stemmed'] = clean_df['text_akhir'].apply(stemmingText)

# 11. Menghapus kata berulang berurutan
clean_df['text_final'] = clean_df['text_akhir'].apply(remove_repeated_words)


## Pelabelan Sentimen Menggunakan Lexicon
Gunakan kamus lexicon positif dan negatif untuk menentukan polaritas sentimen (positif, netral, negatif).


In [6]:
# Langkah 5: Pelabelan Sentimen Menggunakan Lexicon
print("\nLangkah 5: Pelabelan Sentimen Menggunakan Lexicon")

# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_positive[row[0]] = int(row[1])
else:
    print("Failed to fetch positive lexicon data")

# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
if response.status_code == 200:
    reader = csv.reader(StringIO(response.text), delimiter=',')
    for row in reader:
        lexicon_negative[row[0]] = int(row[1])
else:
    print("Failed to fetch negative lexicon data")

# Fungsi untuk menentukan polaritas sentimen
def sentiment_analysis_lexicon_indonesia(text):
    score = 0
    for word in text:
        if word in lexicon_positive:
            score += lexicon_positive[word]
        if word in lexicon_negative:
            score += lexicon_negative[word]

    polarity = ''
    if score >= 7:
        polarity = 'positive'
    elif score == 3:
        polarity = 'neutral'
    else:
        polarity = 'negative'

    return score, polarity

# Terapkan analisis sentimen
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]

print("Distribusi polaritas:")
print(clean_df['polarity'].value_counts())
clean_df.to_csv('processed_ulasan_whatsapp.csv', index=False)



Langkah 5: Pelabelan Sentimen Menggunakan Lexicon
Distribusi polaritas:
polarity
negative    54852
positive     5814
neutral      2229
Name: count, dtype: int64


## Skema Pelatihan
Lakukan tiga skema pelatihan:
- Skema 1: Decision Tree + TF-IDF
- Skema 2: Random Forest + TF-IDF
- Skema 3: Deep Learning (CNN)


In [7]:
print("\nSkema 1: Decision Tree + TF-IDF, Pembagian Data 80/20")
X_train, X_test, y_train, y_test = train_test_split(clean_df['text_final'], clean_df['polarity'], test_size=0.20, random_state=42)

vectorizer_tfidf = TfidfVectorizer(
    max_features=30000,
    ngram_range=(1,2),
    sublinear_tf=True,
    use_idf=True,
)
X_train_tfidf = vectorizer_tfidf.fit_transform(X_train.fillna(""))
X_test_tfidf = vectorizer_tfidf.transform(X_test.fillna(""))

smote = SMOTE(random_state=42)
X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf, y_train)

dt = DecisionTreeClassifier(max_depth=60, min_samples_leaf=2, random_state=42)
dt.fit(X_train_tfidf, y_train)

y_pred_dt_train = dt.predict(X_train_tfidf)
y_pred_dt_test = dt.predict(X_test_tfidf)
accuracy_dt_train = round(accuracy_score(y_train, y_pred_dt_train) * 100)
accuracy_dt_test = round(accuracy_score(y_test, y_pred_dt_test) * 100)
print(f"Akurasi Skema 1 (Decision Tree + TF-IDF) - Training: {accuracy_dt_train}%")
print(f"Akurasi Skema 1 (Decision Tree + TF-IDF) - Testing: {accuracy_dt_test}%")


Skema 1: Decision Tree + TF-IDF, Pembagian Data 80/20
Akurasi Skema 1 (Decision Tree + TF-IDF) - Training: 98%
Akurasi Skema 1 (Decision Tree + TF-IDF) - Testing: 85%


In [8]:
print("\nSkema 2: Random Forest + TF-IDF, Pembagian Data 75/25")
X_train, X_test, y_train, y_test = train_test_split(clean_df['text_final'], clean_df['polarity'], test_size=0.25, random_state=42)

X_train_tfidf = vectorizer_tfidf.fit_transform(X_train.fillna(""))
X_test_tfidf = vectorizer_tfidf.transform(X_test.fillna(""))

X_train_tfidf, y_train = smote.fit_resample(X_train_tfidf, y_train)

rf = RandomForestClassifier(n_estimators=400, max_depth=60, min_samples_leaf=2, random_state=42)
rf.fit(X_train_tfidf, y_train)

y_pred_rf_train = rf.predict(X_train_tfidf)
y_pred_rf_test = rf.predict(X_test_tfidf)
accuracy_rf_train = round(accuracy_score(y_train, y_pred_rf_train) * 100)
accuracy_rf_test = round(accuracy_score(y_test, y_pred_rf_test) * 100)
print(f"Akurasi Skema 2 (Random Forest + TF-IDF) - Training: {accuracy_rf_train}%")
print(f"Akurasi Skema 2 (Random Forest + TF-IDF) - Testing: {accuracy_rf_test}%")


Skema 2: Random Forest + TF-IDF, Pembagian Data 75/25
Akurasi Skema 2 (Random Forest + TF-IDF) - Training: 97%
Akurasi Skema 2 (Random Forest + TF-IDF) - Testing: 89%


In [9]:
print("\nSkema 3: Deep Learning (CNN), Pembagian Data 80/20")

max_words = 10000
max_len = 100

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(clean_df['text_final'])
sequences = tokenizer.texts_to_sequences(clean_df['text_final'])
padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')

label_mapping = {'positive': 0, 'neutral': 1, 'negative': 2}
clean_df['label'] = clean_df['polarity'].map(label_mapping)

X = padded_sequences
y = to_categorical(clean_df['label'], num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Jumlah data training: {len(X_train)}")
print(f"Jumlah data testing: {len(X_test)}")


Skema 3: Deep Learning (CNN), Pembagian Data 80/20
Jumlah data training: 50316
Jumlah data testing: 12579


In [10]:
from tensorflow.python.keras.callbacks import EarlyStopping

embedding_dim = 100
model_cnn = Sequential([
    Embedding(max_words, embedding_dim, input_length=max_len),
    Conv1D(256, 3, activation='relu'),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model_cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_cnn.summary()

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history_cnn = model_cnn.fit(
    X_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[early_stop],
    verbose=1
)

C:\Users\Rucirasatti N\DataspellProjects\latpengml\venv\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 30s 34ms/step - accuracy: 0.8719 - loss: 0.4119 - val_accuracy: 0.9202 - val_loss: 0.2379
Epoch 2/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 26s 32ms/step - accuracy: 0.9245 - loss: 0.2192 - val_accuracy: 0.9238 - val_loss: 0.2248
Epoch 3/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 25s 32ms/step - accuracy: 0.9421 - loss: 0.1628 - val_accuracy: 0.9235 - val_loss: 0.2540
Epoch 4/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 24s 31ms/step - accuracy: 0.9525 - loss: 0.1244 - val_accuracy: 0.9216 - val_loss: 0.2955
Epoch 5/20
787/787 ━━━━━━━━━━━━━━━━━━━━ 24s 30ms/step - accuracy: 0.9567 - loss: 0.0999 - val_accuracy: 0.9222 - val_loss: 0.3049


In [11]:
y_pred_train = model_cnn.predict(X_train)
y_pred_test = model_cnn.predict(X_test)
y_pred_train_labels = np.argmax(y_pred_train, axis=1)
y_pred_test_labels = np.argmax(y_pred_test, axis=1)
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

accuracy_cnn_train = round(accuracy_score(y_train_labels, y_pred_train_labels) * 100)
accuracy_cnn_test = round(accuracy_score(y_test_labels, y_pred_test_labels) * 100)
print(f"Akurasi Skema 3 (CNN) - Training: {accuracy_cnn_train}%")
print(f"Akurasi Skema 3 (CNN) - Testing: {accuracy_cnn_test}%")

1573/1573 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step
394/394 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step
Akurasi Skema 3 (CNN) - Training: 95%
Akurasi Skema 3 (CNN) - Testing: 92%


# Hasil Analisis Sentimen

In [17]:
import pandas as pd

# Dictionary untuk menyimpan hasil akurasi
accuracy_results = {
    'Skema': [
        'Decision Tree + TF-IDF',
        'Random Forest + TF-IDF',
        'CNN'
    ],
    'Akurasi Training (%)': [
        f"{int(accuracy_dt_train)}%",
        f"{int(accuracy_rf_train)}%",
        f"{int(accuracy_cnn_train)}%"
    ],
    'Akurasi Testing (%)': [
        f"{int(accuracy_dt_test)}%",
        f"{int(accuracy_rf_test)}%",
        f"{int(accuracy_cnn_test)}%"
    ]
}

# Buat DataFrame dari dictionary
accuracy_df = pd.DataFrame(accuracy_results)

# Tampilkan tabel
print("\nPerbandingan Akurasi dari Ketiga Skema:")
print(accuracy_df.to_string(index=False))


Perbandingan Akurasi dari Ketiga Skema:
                 Skema Akurasi Training (%) Akurasi Testing (%)
Decision Tree + TF-IDF                  98%                 85%
Random Forest + TF-IDF                  97%                 89%
                   CNN                  95%                 92%


## Inference
Lakukan inference pada beberapa sampel data untuk melihat hasil prediksi dari model.

In [21]:
# Langkah 7: Inference untuk Testing
print("\nLangkah 7: Inference untuk Testing (Menggunakan Model Skema 2 - Random Forest)")
# Ambil 5 sampel acak dari data testing untuk inference
sample_indices = np.random.choice(X_test.shape[0], 5, replace=False)
sample_texts = X_test[sample_indices]
sample_labels = y_test[sample_indices]

# Untuk Random Forest, kita perlu mengubah kembali teks ke format TF-IDF
sample_texts_raw = clean_df['text_final'].iloc[sample_indices]
sample_texts_tfidf = vectorizer_tfidf.transform(sample_texts_raw.fillna(""))

# Prediksi menggunakan model Random Forest (Skema 2)
predictions_rf = rf.predict(sample_texts_tfidf)
true_labels = np.argmax(sample_labels, axis=1)

# Mapping kembali ke label kategorikal
reverse_label_mapping = {0: 'positive', 1: 'neutral', 2: 'negative'}
predicted_labels_text = [predictions_rf[i] for i in range(len(predictions_rf))]
true_labels_text = [reverse_label_mapping[label] for label in true_labels]


Langkah 7: Inference untuk Testing (Menggunakan Model Skema 2 - Random Forest)


In [22]:
# Tampilkan hasil inference
print("\nHasil Inference (Random Forest):")
for i in range(len(sample_texts_raw)):
    print(f"Sampel {i+1}:")
    print(f"Teks: {sample_texts_raw.iloc[i]}")
    print(f"Prediksi: {predicted_labels_text[i]}")
    print(f"Label Sebenarnya: {true_labels_text[i]}\n")


Hasil Inference (Random Forest):
Sampel 1:
Teks: senang mengunakan whatsap kena spam nomer masukan salah tidak mengunakan whatsap tergantung november februari
Prediksi: negative
Label Sebenarnya: positive

Sampel 2:
Teks: aplikasi nya dijadikan perangkat pendamping eror tidak login cek koneksi internet normal sped mbs
Prediksi: negative
Label Sebenarnya: negative

Sampel 3:
Teks: tolong pengiriman video status sesuai kualitas kamera bikin cerita wa gambar video pecah tolong perbaiki
Prediksi: negative
Label Sebenarnya: negative

Sampel 4:
Teks: bagus temen tidak nge priv p nya tidak p kalo status burem tidak jelasterus permasalahan voice note teriak ngomong deketin kekipas tetep aja tidak suara jg temen kadang gitu tolong diperbaiki cape whatsap fitur ngebug
Prediksi: negative
Label Sebenarnya: negative

Sampel 5:
Teks: perbaharuan handphone skli pdhal otomatis lerbaharui aja slu
Prediksi: negative
Label Sebenarnya: negative



In [23]:
# Inference menggunakan model CNN (Skema 3)
print("\nLangkah 7: Inference untuk Testing (Menggunakan Model Skema 3 - CNN)")
predictions_cnn = model_cnn.predict(sample_texts)
predicted_labels_cnn = np.argmax(predictions_cnn, axis=1)
predicted_labels_cnn_text = [reverse_label_mapping[label] for label in predicted_labels_cnn]

# Tampilkan hasil inference
print("\nHasil Inference (CNN):")
for i in range(len(sample_texts_raw)):
    print(f"Sampel {i+1}:")
    print(f"Teks: {sample_texts_raw.iloc[i]}")
    print(f"Prediksi: {predicted_labels_cnn_text[i]}")
    print(f"Label Sebenarnya: {true_labels_text[i]}\n")

print("\nProses selesai!")


Langkah 7: Inference untuk Testing (Menggunakan Model Skema 3 - CNN)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step

Hasil Inference (CNN):
Sampel 1:
Teks: senang mengunakan whatsap kena spam nomer masukan salah tidak mengunakan whatsap tergantung november februari
Prediksi: positive
Label Sebenarnya: positive

Sampel 2:
Teks: aplikasi nya dijadikan perangkat pendamping eror tidak login cek koneksi internet normal sped mbs
Prediksi: negative
Label Sebenarnya: negative

Sampel 3:
Teks: tolong pengiriman video status sesuai kualitas kamera bikin cerita wa gambar video pecah tolong perbaiki
Prediksi: negative
Label Sebenarnya: negative

Sampel 4:
Teks: bagus temen tidak nge priv p nya tidak p kalo status burem tidak jelasterus permasalahan voice note teriak ngomong deketin kekipas tetep aja tidak suara jg temen kadang gitu tolong diperbaiki cape whatsap fitur ngebug
Prediksi: negative
Label Sebenarnya: negative

Sampel 5:
Teks: perbaharuan handphone skli pdhal otomatis lerbaharui aja slu
Prediks